# **Projet portant sur les réseaux de neurones**
### **Réalisé par Fréjoux Gaëtan et Niord Mathieu**
---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Développement d'un perceptron
α sera égal à 0.1

### 1.1. Mise en place d'un perceptron simple

* *Créer la fonction `perceptron_simple(x, w, active)`*

In [ ]:
ALPHA = 0.1  # learning rate

def perceptron_simple(x, w, active):
  seuil = w[0]
  dot = np.dot(x, w[1:])
  x = seuil + dot
  return np.sign(x) if (active == 0) else np.tanh(x)


def plot_with_class(X, weight, c, title, min_y, max_y):
    x = np.linspace(min_y, max_y)
    y = (weight[0] + x*weight[1]) / (-weight[2])
    
    plt.plot(x, y)
    plt.title(title)
    plt.scatter(X[:, 0], X[:, 1], c=c)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.grid()
    plt.xlim(min_y, max_y)
    plt.ylim(min_y, max_y) 
    plt.show()

**[Question du formulaire] : Donner quelques éléments de commentaires sur la stratégie que vous avez utilisée pour 
développer le perceptron simple**

**[Réponse]** : Nous avons utilisé la fonction `np.dot` pour calculer le produit scalaire entre les vecteurs `x` et `w`. Ensuite nous avons ajouté le seuil à ce produit scalaire. Enfin nous avons utilisé la valeur de `active` pour obtenir la sortie du perceptron avec différentes fonctions d'activation (ici `np.tanh` et `np.sign`).

* *Tester votre perceptron avec l'exemple du OU logique vu en cours (en utilisant la fonction `sign(x)` comme fonction d'activation)*

In [ ]:
W_OR = np.array([-0.5, 1, 1])
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])

Result_OR = perceptron_simple(X, W_OR, 0) # TODO: fix this pcq j'ai pas compris son truc avec sign(x)
print(Result_OR)

* *Afficher dans le cadre de l'exemple du OU logique sur la même figure les différents éléments de l'ensemble d'apprentissage et la droite séparatrice associée aux poids du neurone sur la même figure*

In [ ]:
plot_with_class(X, W_OR, Result_OR, "1.1 - Classification par perceptron simple sur un ensemble de données OR", -1, 2)

**[Question du formulaire] : Montrer que votre code fonctionne dans le cas du OU**

**[Réponse]** : On observe ci-dessus que le perceptron fonctionne bien pour le OU logique.

**[Question du formulaire] : Afficher la figure individu/frontière et commenter**

**[Réponse]** : On observe que la droite séparatrice est bien la droite qui sépare les deux classes. D'un côté on a les individus dont la sortie est 1 et de l'autre les individus dont la sortie est -1. Dans le cas du OU logique, [0,0] est associé à -1 et [0,1], [1,0] et [1,1] sont associés à 1.

### 1.2. Etude de l'apprentissage

#### 1.2.1. Programmation apprentissage Widrow-Hoff

* *Créer la fonction `apprentissage_widrow(x, yd, epoch, batch_size)`. La droite séparatrice et les points d'apprentissage doivent être affichés à chaque itération (une itération correspond à la présentation de tous les individus de l'ensemble d'apprentissage), ains que l'erreur de classification*

In [ ]:
def apprentissage_widrow(x, yd, epoch, batch_size):

    w = np.random.randn(3)
    errors = []
    for i in range(epoch):
        w_temp = w
        errors.append(0)
        for j in range(len(x)):

            y = perceptron_simple(x[j], w, 1)  # with tanh
            r = - (yd[j] - y) * (1 - y * y)

            w_temp += ALPHA * r * np.array([1, x[j][0], x[j][1]])

            errors[i] += r**2

            if (j % batch_size) == 0: 
                w = w_temp
        print("Epoch ", i + 1, " : ", errors[i]) # Errr value display
        
        if (i % (epoch // 10) == 0): #allow to show only 10 plots at max
            plot_with_class(x, w, yd, f"Apprentissage_widrow epoch {i}", min(x[:,0]) - 0.5, max(x[:,0]) + 0.5)

        if (errors[i] == 0 or (i != 0 and (errors[i - 1] - errors[i] == 0))): break
        
    return w, errors

**[Question du formulaire] : Donner quelques éléments de commentaires sur la stratégie que vous avez utilisée pour 
développer l’apprentissage Widrow**

**[Réponse]** : Nous avons tout d'abord initialisé les poids aléatoirement. Ensuite nous avons parcouru les données d'apprentissage `epoch` fois. Pour chaque individu, nous avons calculé la sortie du perceptron et nous avons mis à jour les poids en fonction de l'erreur tout les `batch_size` individus. Enfin, pour suivre l'évolution de l'apprentissage, nous avons affiché la droite séparatrice tous les 10% de l'ensemble d'apprentissage. Nous avons également affiché l'erreur de classification à chaque itération.

#### 1.2.2. Test 1 simple

* *Charger les données p2_d1.txt*

In [ ]:
Data1 = np.loadtxt("res/p2_d1.txt")
CLASSIF = [1]*25 + [-1]*25
plt.scatter(Data1[0, :25], Data1[1, :25], c='r')
plt.scatter(Data1[0, 25:], Data1[1, 25:], c='b')
plt.legend(['Classe 1', 'Classe 2'])

* *Appliquer l'algorithme d'apprentissage sur les données. Afficher l'évolution de l'erreur. Vérifier que la frontière est correcte*

In [ ]:
w1, erreur1 = apprentissage_widrow(Data1.T, CLASSIF, 500, 25)
print('W1 : ', w1)

**[Question du formulaire] : Indiquez graphiquement quelques étapes de l’apprentissage (ensemble et droite de 
séparation)**

**[Réponse]** : Voir ci-dessus.

**[Question du formulaire] : Représenter l’erreur en fonction des itérations**

In [ ]:
plt.plot(erreur1)

**[Réponse]** : Voir ci-dessus.

**[Question du formulaire] : Faites quelques tests avec des initialisations différentes, commenter le résultat après 
convergence**

**[Réponse]** : On observe que l'erreur de classification tend vers 0. On remarque également que la droite séparatrice est bien la droite qui sépare les deux classes.

#### 1.2.3. Test 2

* *Charger les données p2_d2.txt*

In [ ]:
Data2 = np.loadtxt("res/p2_d2.txt")
plt.scatter(Data2[0, :25], Data2[1, :25], c='r')
plt.scatter(Data2[0, 25:], Data2[1, 25:], c='b')
plt.legend(['Classe 1', 'Classe 2'])

* *Appliquer l'algorithme d'apprentissage sur les données. Afficher l'évolution de l'erreur. Vérifier que la frontière est correcte*

In [ ]:
w2, erreur2 = apprentissage_widrow(Data2.T, CLASSIF, 500, 25)
print('W2 : ', w2)

**[Question du formulaire] : Indiquez graphiquement quelques étapes de l’apprentissage (ensemble et droite de 
séparation)**


**[Réponse]** : Voir ci-dessus.

**[Question du formulaire] : Représenter l’erreur en fonction des itérations, Comparer avec le test précédent**

In [ ]:
plt.plot(erreur2)

**[Réponse]** : On observe pour ce deuxième échantillon que l'erreur de classification ne tend pas vers 0. En effet, il y a des individus qui sont mal classés. Dû à la nature des données, il est difficile de trouver une droite séparatrice qui sépare les deux classes. Pour autant, on observe que la droite séparatrice est bien la droite qui sépare les deux classes au mieux. Il est également intéressant de noter que l'erreur réaugmente pendant un certain temps puis diminue à nouveau cette fois-ci vers 0.

**[Question du formulaire] : Faites quelques tests avec des initialisations différentes, commenter le résultat après 
convergence**

**[Réponse]** : Voir ci-dessus.

### 1.3 Perceptron multicouches

#### 1.3.1. Mise en place d'un perceptron multicouche

* *Créer la fonction `multiperceptron(x, w1, w2)`*

In [ ]:


def multiperceptron(x, w1, w2):
    
    def activation(x): 
        return 1 / (1 + np.exp(-x))

    x = np.array([1, x[0], x[1]])

    u1 = np.dot(w1[:, 0], x)
    u2 = np.dot(w1[:, 1], x)

    y1 = activation(u1)
    y2 = activation(u2)

    uf = np.dot(w2, np.array([1, y1, y2]))
    
    yf = activation(uf)

    return [y1, y2], yf


**[Question du formulaire] : Donner quelques éléments de commentaires sur la stratégie que vous avez utilisée pour 
développer le perceptron Multicouche**

**[Réponse]** : Nous avons tout d'abord créé une fonction `sigmoid` qui prend en paramètre un vecteur et qui renvoie le vecteur des valeurs de la fonction sigmoïde appliquée à chaque élément du vecteur. Ensuite nous avons créé la fonction `multiperceptron` qui prend en paramètre un vecteur `x` et deux matrices `w1` et `w2`. Cette fonction renvoie la sortie du perceptron multicouche. Pour cela, nous avons tout d'abord traversé la couche cachée en appliquant la fonction sigmoïde à la multiplication de `x` et `w1`. Ensuite nous avons traversé la couche de sortie en appliquant la fonction sigmoïde à la multiplication du résultat précédent et `w2`. Enfin, nous avons renvoyé le résultat de la couche de sortie.

* *Tester votre perceptron multicouches avec l'exemple ci-dessous pour un entrée x = [1 1]' :*  
  
<img src="exemple_1_3_1.jpg" width="600">

In [ ]:
x = np.array([1, 1])
w1 = np.array([[-.5, .5], [2., .5], [-1., 1.]])
w2 = np.array([2., -1., 1.])

print(multiperceptron(x, w1, w2))

**[Question du formulaire] : Indiquer le résultat numérique et par calcul (en donnant le détail) pour le test demandé**

**[Réponse]** : On observe ci-dessus que le résultat est environ égal à 0.91. Pour le calcul, on a :

u1 = 1 * -0.5 + 1 * 2 + 1 * -1 = 0.5  
y1 = 1 / (1 + exp(-0.5)) = 0.62
 
u2 = 1 * 0.5 + 1 * 0.5 + 1 * 1 = 2  
y2 = 1 / (1 + exp(-2)) = 0.88

uf = 1 * 2 + 0.62 * -1 + 0.88 * 1 = 2.26  
yf = 1 / (1 + exp(-uf)) = 0.91

#### 1.3.2. Programmation apprentissage multicouches

* *Créer une fonction `multiperceptron_widrow(x, yd, epoch, batch_size)`*

In [ ]:
ALPHA = 0.5

def multiperceptron_widrow(x, yd, epoch, batch_size):
    def derivative(x): 
        return x * (1 - x)

    w1 = np.random.rand(3, 2) - 0.5
    w2 = np.random.rand(3) - 0.5
    new_w1 = np.zeros((3, 2))
    new_w2 = np.zeros((3))
    errors = np.zeros((epoch))

    for i in range(epoch):
        for j in range(x.shape[1]):
            indiv = x[:, j]
            target = yd[j]

            y1_predict, yf_predict = multiperceptron(indiv, w1, w2)

            errors[i] += (target - yf_predict)**2

            rf = -(target - yf_predict) * derivative(yf_predict)

            r11 = w2[1] * rf * derivative(y1_predict[0])
            r12 = w2[2] * rf * derivative(y1_predict[1])

            indiv = np.array([1, indiv[0], indiv[1]])

            new_w1[:, 0] += - ALPHA * r11 * indiv
            new_w1[:, 1] += - ALPHA * r12 * indiv

            y1_predict = np.array([1, y1_predict[0], y1_predict[1]])

            new_w2 += - ALPHA * rf * y1_predict

            if j % batch_size == 0:
                w1 += new_w1
                w2 += new_w2
                new_w1 = np.zeros((3, 2))
                new_w2 = np.zeros((3))
                
        error = round(errors[i], 3)

        print("epoch", i + 1, " : ", error)

        if error < 0.01:
            break

    return w1, w2, errors

**[Question du formulaire] : Donner quelques éléments de commentaires sur la stratégie que vous avez utilisée pour 
développer l’apprentissage Multicouche**

**[Réponse]** : Nous avons tout d'abord créé une fonction `multiperceptron_widrow` qui prend en paramètre un vecteur `x`, un vecteur `yd`, un entier `epoch` et un entier `batch_size`. Cette fonction renvoie les matrices `w1` et `w2` et `errors` après apprentissage. Pour cela, nous avons tout d'abord créé les matrices `w1` et `w2` aléatoirement. Ensuite nous avons créé une boucle qui va s'exécuter `epoch` fois. Dans cette boucle, nous passons par chaque individu de l'échantillon `x`. Pour chaque individu, nous calculons la sortie du perceptron multicouche. Ensuite nous calculons l'erreur de classification. Enfin nous mettons à jour les matrices `w1` et `w2` tous les `batch_size` individus. Enfin nous renvoyons les matrices `w1` et `w2` et `errors`.

* *Créer l'ensemble d'apprentissage*

In [ ]:
x = np.array([[0, 1, 0, 1], [0, 0, 1, 1]])
yd = np.array([0, 1, 1, 0])
epoch = 20000

* *Afficher cet ensemble avec la fonction affiche_classe*

In [ ]:
plt.scatter(x[0, yd == 0], x[1, yd == 0], c='r')
plt.scatter(x[0, yd == 1], x[1, yd == 1], c='b')
plt.legend(['False', 'True'])

* *Pensez-vous que ce problème puisse être traité par un perceptron simple ?*

Nous ne pouvons pas traiter ce problème avec un perceptron simple car il n'est pas linéairement séparable. En effet, il n'existe pas de droite qui sépare les deux classes.

* *Appliquer votre algorithme d'apprentissage*

In [ ]:
w1, w2, erreur = multiperceptron_widrow(x, yd, epoch, 4)
print(erreur)

**[Question du formulaire] : Représenter l’erreur en fonction des itérations. Commenter**

In [ ]:
plt.plot(erreur)

**[Réponse]** : Voir ci-dessus.

* *Tester, à partir de votre fonction `multiperceptron`, le réseau de neurones ainsi obtenu sur l'ensemble d'apprentissage*

In [ ]:
print(multiperceptron(np.array([0, 0]), w1, w2))
print(multiperceptron(np.array([0, 1]), w1, w2))
print(multiperceptron(np.array([1, 0]), w1, w2))
print(multiperceptron(np.array([1, 1]), w1, w2))

**[Question du formulaire] : Tester votre structure après apprentissage et montrer que c’est bien un XOR**

**[Réponse]** : On obser ci-dessus que le résultat est bien un XOR. En effet, on a :  
pour x = [0 0] : y = 0  
pour x = [0 1] : y = 1  
pour x = [1 0] : y = 1  
pour x = [1 1] : y = 0  

* *Afficher les droites séparatrices associées aux différents neurones et les points de l'ensemble d'apprentissage*

In [ ]:
x = np.linspace(-2, 2)

y = (w1[0, 0] + x*w1[1, 0]) / (-w1[2, 0])
y2 = (w1[0, 1] + x*w1[1, 1]) / (-w1[2, 1])
y3 = (w2[0] + x*w2[1]) / (-w2[2])


plt.plot(x, y)
plt.plot(x, y2)
plt.plot(x, y3)
plt.scatter(X[:, 0], X[:, 1])
plt.legend(['P1', 'P2', 'P3', 'Data'])
plt.grid()
plt.show()

**[Question du formulaire] : Représenter les trois droites séparatrices et l’ensemble d’apprentissage. Expliquer le 
fonctionnement**

**[Réponse]** : Voir ci-dessus. On observe bien que les droites permettent de séparer les deux classes.

## 2. Deep et Full-connected : discrimination d'une image

### 2.1. Approche basée Descripteurs (basé modèle)

#### 2.1.1. Calcul des descripteurs

* *Importer les différents tableaux de mesure et créer un vecteur de label indiquant la classe sous forme d'un chiffre de chauqe image*

In [ ]:
#TODO

#### 2.1.2. Mise en place d'un système de discrimination basé structure Full-Connected

* *Mettre en place un système de discrimination qui pour la présentation d'une image inconnue et de son vecteur de mesures associé propose une classe*

* *Tester la procédure de la classification avec les 5 types de mesures et différentes images incoonues*

In [ ]:
#TODO

* *Analyser les résultats (matrice de confusion, taux d'erreur)*

In [ ]:
#TODO

TODO commentaires

* *Comparer avec différents hyperparamètres ????(nombre de neurones, nombre de couches, nombre d'itérations, taille du batch)????*

In [ ]:
#TODO

TODO commentaires

#### 2.1.3. Approche "Deep" (basée Data)

* *Mettre en place un système de classification qui a en données une image, calcul les descripteurs par des couches de convolution*

In [ ]:
#TODO

* *Tester des structures simples*

* *Etudier l'influence des paramètres*

* *Etudier l'évolution de la fonction de coût*

* *Comparer avec les résultats avec les méthodes basées caractéristiques*

* *Tester avec des structures plus complexes. La "Data augmentation" doit probablement être utilisée*

In [ ]:
#TODO